In [1]:
!pip install -q medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import numpy as np
import pandas as pd
import os
import medmnist
from medmnist import INFO, Evaluator
import torch
from torch.utils.data import Dataset, Subset, DataLoader
from pathlib import Path
import random
import timm
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torch.nn.utils import clip_grad_norm_
from sklearn.metrics import f1_score, recall_score  # recall_score 추가
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
from collections import defaultdict

# --- Configuration ---
config = {
    'backbone': 'convnext_tiny.in12k_ft_in1k',
    'pretrained': True,
    'batch_size': 512,
    'num_epochs': 30,
    'lr': 1e-3,
    'optimizer': 'AdamW',
    'weight_decay': 0.05,
    'gradient_clip_norm': 1.0,
    'early_stopping_patience': 7,
    'scheduler': 'onecycle',
    'num_workers': 4,
    'use_amp': True,
    'head_type': 'bottleneck',
    'dropout_rate': 0.2,
    'stochastic_depth_rate': 0.1,
    'use_focal_loss': False,  # Focal Loss 사용 여부
    'focal_loss_gamma': 2.0, # Focal Loss 감마 값
    'early_stopping_target_task': 'breastmnist',  # 조기 종료 기준 작업
    'early_stopping_metric': 'recall',  # 조기 종료 기준 지표 (recall 또는 f1)
}

DEBUG = False  # Set to True for faster debugging (fewer epochs/data)
if DEBUG:
    config['num_epochs'] = 1
    config['batch_size'] = 64

# --- Data Loading ---
def load_medmnist_from_npz(data_flag, debug=False):
    data_path = Path('/kaggle/input/tensor-reloaded-multi-task-med-mnist/data') / f'{data_flag}.npz'
    data = np.load(data_path)
    info = INFO[data_flag]
    n_classes = len(info['label'])

    class NPZDataset(Dataset):
        def __init__(self, images, labels=None):
            self.images = images
            self.labels = labels

        def __len__(self):
            return len(self.images)

        def __getitem__(self, idx):
            image = self.images[idx]
            label = self.labels[idx] if self.labels is not None else None
            return image, label  # Return NumPy arrays

    if debug:
        # Use a smaller subset of the training data for debugging
        train_size = len(data['train_images'])
        indices = list(range(train_size))
        random.seed(42)  # For reproducibility
        debug_size = int(0.1 * train_size)  # Use 10% of the data
        debug_indices = random.sample(indices, debug_size)

        train_images = data['train_images'][debug_indices]
        train_labels = data['train_labels'][debug_indices] if 'train_labels' in data else None
        train_dataset = NPZDataset(train_images, train_labels)
    else:
        train_dataset = NPZDataset(data['train_images'], data.get('train_labels'))

    val_dataset = NPZDataset(data['val_images'], data.get('val_labels'))
    test_dataset = NPZDataset(data['test_images'], data.get('test_labels'))  # Include test labels

    return train_dataset, val_dataset, test_dataset, info

DATASETS = [
    'pathmnist', 'dermamnist', 'octmnist', 'pneumoniamnist', 'retinamnist',
    'breastmnist', 'bloodmnist', 'tissuemnist', 'organamnist', 'organcmnist',
    'organsmnist'
]

def load_all_datasets(debug=False):
    datasets = {}
    for data_flag in DATASETS:
        print(f"Loading {data_flag}...")
        train, val, test, info = load_medmnist_from_npz(data_flag, debug=debug)
        datasets[data_flag] = {
            'train': train,
            'val': val,
            'test': test,
            'info': info
        }
    return datasets

def calculate_class_weights(datasets):
    weights = {}
    for data_flag in DATASETS:
        labels = datasets[data_flag]['train'].labels
        if labels is None:  # Handle cases where labels are missing
            weights[data_flag] = None
            continue
        if isinstance(labels, torch.Tensor):
            labels = labels.numpy()  # Convert to NumPy array if needed
        num_classes = len(datasets[data_flag]['info']['label'])
        class_counts = np.bincount(labels.flatten(), minlength=num_classes)
        total = class_counts.sum()
        raw_weights = total / (class_counts + 1e-6)  # Avoid division by zero
        normalized_weights = raw_weights / raw_weights.mean()  # Normalize
        weights[data_flag] = torch.FloatTensor(normalized_weights)
    return weights

# --- Model Definition ---
class SelfAttention(nn.Module):
    """Simple self-attention layer."""
    def __init__(self, dim, heads=8, dropout=0.0):
        super().__init__()
        self.heads = heads
        self.scale = dim ** -0.5  # Scaled dot-product attention
        self.to_qkv = nn.Linear(dim, dim * 3, bias=False)  # Learnable projections
        self.attend = nn.Softmax(dim=-1)  # Softmax for attention weights
        self.dropout = nn.Dropout(dropout)
        self.to_out = nn.Sequential(nn.Linear(dim, dim), nn.Dropout(dropout))

    def forward(self, x):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim=-1)  # Split into Q, K, V
        q, k, v = map(lambda t: t.reshape(b, n, h, -1).permute(0, 2, 1, 3), qkv)
        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale  # Scaled dot product
        attn = self.attend(dots)
        attn = self.dropout(attn)
        out = torch.matmul(attn, v)  # Weighted sum of values
        out = out.permute(0, 2, 1, 3).reshape(b, n, -1)  # Concatenate heads
        return self.to_out(out)

class MedMNISTMultiTaskModel(nn.Module):
    def __init__(self, backbone_name, pretrained, head_type, dropout_rate, stochastic_depth_rate):
        super().__init__()
        self.task_outputs = {d: len(INFO[d]['label']) for d in DATASETS}
        self.backbone = timm.create_model(
            backbone_name, pretrained=pretrained, num_classes=0, drop_path_rate=stochastic_depth_rate
        )
        # Adapt the input layer to handle potential grayscale images:
        self.backbone.stem[0] = nn.Conv2d(3, self.backbone.stem[0].out_channels, kernel_size=3, stride=1, padding=1)
        feat_dim = self.backbone.num_features  # Get feature dimension from backbone

        self.heads = nn.ModuleDict()
        for task, num_classes in self.task_outputs.items():
            if head_type == 'simple':
                head = nn.Sequential(
                    nn.LayerNorm(feat_dim),
                    nn.Linear(feat_dim, num_classes)
                )
            elif head_type == 'bottleneck':
                head = nn.Sequential(
                    nn.LayerNorm(feat_dim),
                    nn.Linear(feat_dim, feat_dim // 4),
                    nn.GELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(feat_dim // 4, num_classes)
                )
            elif head_type == 'attention':
                head = nn.Sequential(
                    nn.LayerNorm(feat_dim),
                    nn.Linear(feat_dim, feat_dim//2),
                    nn.GELU(),
                    SelfAttention(feat_dim//2, heads=4, dropout=dropout_rate),
                    nn.Linear(feat_dim // 2, num_classes)
                )
            else:
                raise ValueError(f"Invalid head_type: {head_type}")
            self.heads[task] = head

    def forward(self, x, task_ids=None):
        features = self.backbone(x)
        if task_ids is not None:  # Training/validation: task-specific outputs
            outputs = torch.zeros(len(task_ids), max(self.task_outputs.values())).to(features.device)
            for i, task_id in enumerate(task_ids):
                task_name = DATASETS[task_id]
                task_output = self.heads[task_name](features[i:i+1])
                outputs[i, :self.task_outputs[task_name]] = task_output.squeeze(0)
            return outputs
        else:  # Potentially for inference/feature extraction
            return {task: head(features) for task, head in self.heads.items()}

# --- Dataset ---
class MedMNISTMultiDataset(Dataset):
    def __init__(self, datasets, split='train', transform=None):
        super().__init__()
        self.datasets = datasets
        self.split = split
        self.transform = transform
        self.dataset_indices = []
        # Create a combined list of (dataset_index, sample_index)
        for dataset_idx, (name, dataset_dict) in enumerate(datasets.items()):
            dataset = dataset_dict[split]
            n_samples = len(dataset)
            self.dataset_indices.extend([(dataset_idx, i) for i in range(n_samples)])

    def __len__(self):
        return len(self.dataset_indices)

    def __getitem__(self, idx):
        dataset_idx, sample_idx = self.dataset_indices[idx]
        dataset_name = DATASETS[dataset_idx]
        dataset = self.datasets[dataset_name][self.split]
        image, label = dataset[sample_idx]  # Get NumPy arrays

        # Convert NumPy array to PIL Image
        if isinstance(image, np.ndarray):
            # Handle both grayscale (H, W) and color (H, W, C) images
            if image.ndim == 2:  # Grayscale
                image = transforms.functional.to_pil_image(image, mode='L') # Explicitly specify mode
            elif image.ndim == 3:
                 # Transpose if channels are first
                if image.shape[0] in [1, 3]:
                    image = image.transpose(1, 2, 0)
                image = transforms.functional.to_pil_image(image)


        if self.transform:
            image = self.transform(image)  # Apply transforms

        # Convert label to tensor if it exists
        if label is not None:
            label = torch.tensor(label, dtype=torch.long)
        else:
            label = torch.tensor(-1, dtype=torch.long)  # Dummy label for test set

        return image, label, torch.tensor(dataset_idx, dtype=torch.long)  # Return task ID

# --- Trainer ---
class Trainer:
    def __init__(self, model, train_dataset, val_dataset, config, device='cuda'):
        self.model = model.to(device)
        self.device = device
        self.config = config
        self.scaler = GradScaler(enabled=config['use_amp']) # Use standard GradScaler
        # 조기 종료 기준 관련 변수 초기화
        if self.config['early_stopping_metric'] == 'recall':
          self.best_val_metric = 0.0
        elif self.config["early_stopping_metric"] == "f1":
          self.best_val_metric = 0.0  # Assuming higher is better (e.g., F1 score)
        self.early_stopping_counter = 0
        self.train_loader = DataLoader(
            train_dataset, batch_size=config['batch_size'], shuffle=True,
            num_workers=config['num_workers'], pin_memory=True, persistent_workers=True
        )
        self.val_loader = DataLoader(
            val_dataset, batch_size=config['batch_size'], shuffle=False,
            num_workers=config['num_workers'], pin_memory=True, persistent_workers=True
        )
        self.class_weights = calculate_class_weights(datasets)

        # Optimizer
        if config['optimizer'] == 'AdamW':
            self.optimizer = AdamW(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
        else:
            raise ValueError(f"Invalid optimizer: {config['optimizer']}")

        # Scheduler
        if config['scheduler'] == 'onecycle':
            self.scheduler = OneCycleLR(
                self.optimizer, max_lr=config['lr'], epochs=config['num_epochs'],
                steps_per_epoch=len(self.train_loader)
            )
        else:
            raise ValueError(f"Invalid scheduler: {config['scheduler']}")

        # Loss function (Focal Loss or CrossEntropyLoss)
        if config['use_focal_loss']:
            self.criterion = self.focal_loss  # Use the defined focal_loss method
        else:
            self.criterion = torch.nn.CrossEntropyLoss() # Default criterion

    # Focal Loss implementation (as a method within Trainer)
    def focal_loss(self, outputs, labels, class_weights=None):
        gamma = self.config['focal_loss_gamma']
        ce_loss = F.cross_entropy(outputs, labels, reduction='none', weight=class_weights)
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt)**gamma * ce_loss
        return focal_loss.mean()

    def train_epoch(self):
        self.model.train()
        total_loss = 0
        task_predictions = {task: {'preds': [], 'targets': []} for task in DATASETS}
        pbar = tqdm(self.train_loader, desc='Training', dynamic_ncols=True)

        for images, labels, task_ids in pbar:
            images, labels, task_ids = images.to(self.device), labels.to(self.device), task_ids.to(self.device)

            with autocast(enabled=self.config['use_amp']):
                outputs = self.model(images, task_ids)
                losses = []
                for i, (output, label, task_id) in enumerate(zip(outputs, labels, task_ids)):
                    task_name = DATASETS[task_id]
                    num_classes = self.model.task_outputs[task_name]
                    task_output = output[:num_classes].unsqueeze(0)  # Ensure correct shape
                    task_label = label.view(-1) # Ensure correct shape

                    class_weight = self.class_weights.get(task_name)
                    if class_weight is not None:
                        class_weight = class_weight.to(self.device)

                    # Use Focal Loss if enabled, otherwise use CrossEntropyLoss
                    if self.config['use_focal_loss']:
                      loss = self.focal_loss(task_output, task_label, class_weight)
                    else:
                      loss_fn = torch.nn.CrossEntropyLoss(weight=class_weight)
                      loss = loss_fn(task_output, task_label)

                    losses.append(loss)

                loss = torch.stack(losses).mean() # Use mean for multi-task loss

            self.optimizer.zero_grad()
            self.scaler.scale(loss).backward()
            clip_grad_norm_(self.model.parameters(), self.config['gradient_clip_norm'])
            self.scaler.step(self.optimizer)
            self.scaler.update()
            self.scheduler.step()

            total_loss += loss.item()
            pbar.set_postfix({'loss': total_loss / (pbar.n + 1)})

            for i, (output, label, task_id) in enumerate(zip(outputs, labels, task_ids)):
                task_name = DATASETS[task_id]
                num_classes = self.model.task_outputs[task_name]
                pred = output[:num_classes].argmax(dim=0).cpu().item()
                target = label.item()
                task_predictions[task_name]['preds'].append(pred)
                task_predictions[task_name]['targets'].append(target)

        # Calculate F1 scores for each task
        task_f1_scores = {
            task: f1_score(task_predictions[task]['targets'], task_predictions[task]['preds'], average='macro')
            if len(task_predictions[task]['preds']) > 0 else 0.0 for task in DATASETS
        }
        return total_loss / len(self.train_loader), task_f1_scores

    @torch.no_grad()
    def validate(self):
        self.model.eval()
        total_loss = 0
        task_predictions = {task: {'preds': [], 'targets': []} for task in DATASETS}

        pbar = tqdm(self.val_loader, desc='Validating', dynamic_ncols=True)
        for images, labels, task_ids in pbar:
            images, labels, task_ids = images.to(self.device), labels.to(self.device), task_ids.to(self.device)
            labels = labels.view(-1).long()  # Flatten labels
            outputs = self.model(images, task_ids)
            losses = []

            for task_name in set(DATASETS[tid.item()] for tid in task_ids):  # Iterate through unique tasks
                task_mask = torch.tensor([DATASETS[tid.item()] == task_name for tid in task_ids], device=self.device)
                if not task_mask.any():  # Skip if no samples for this task in batch
                    continue
                task_outputs = outputs[task_mask]
                task_labels = labels[task_mask]
                n_classes = self.model.task_outputs[task_name]

                class_weight = self.class_weights.get(task_name)  # Use .get()
                if class_weight is not None:
                    class_weight = class_weight.to(self.device)

                # Use Focal Loss if enabled
                if self.config['use_focal_loss']:
                    task_loss = self.focal_loss(task_outputs[:, :n_classes], task_labels, class_weight)
                else:
                    loss_fn = torch.nn.CrossEntropyLoss(weight=class_weight)
                    task_loss = loss_fn(task_outputs[:, :n_classes], task_labels)

                losses.append(task_loss)

            loss = torch.stack(losses).mean() # Use mean for multi-task loss
            total_loss += loss.item()

            for i, (task_id, label) in enumerate(zip(task_ids, labels)):
                task_name = DATASETS[task_id.item()]
                n_classes = self.model.task_outputs[task_name]
                pred = outputs[i, :n_classes].argmax(dim=0).cpu().item()
                task_predictions[task_name]['preds'].append(pred)
                task_predictions[task_name]['targets'].append(label.cpu().item())

        # Calculate F1 scores and Recall for each task.
        task_f1_scores = {}
        task_recall_scores = {} # recall 추가
        for task in DATASETS:
            if len(task_predictions[task]['preds']) > 0:
                task_f1_scores[task] = f1_score(task_predictions[task]['targets'], task_predictions[task]['preds'], average='macro')
                task_recall_scores[task] = recall_score(task_predictions[task]['targets'], task_predictions[task]['preds'], average='macro') # recall 계산
            else:
                task_f1_scores[task] = 0.0
                task_recall_scores[task] = 0.0

        # 조기 종료 metric 계산
        if self.config['early_stopping_metric'] == 'recall':
          target_task = self.config['early_stopping_target_task']
          val_metric = task_recall_scores[target_task] if target_task in task_recall_scores else 0.0

        elif self.config['early_stopping_metric'] == 'f1':
            # Compute the *harmonic mean* of the F1 scores.
            val_f1_values = list(task_f1_scores.values())
            val_metric = len(val_f1_values) / sum(1 / f1 if f1 > 0 else 1e-6 for f1 in val_f1_values)

        return total_loss / len(self.val_loader), val_metric, task_f1_scores, task_recall_scores  # Return individual task scores


    def train(self):
        for epoch in range(self.config['num_epochs']):
            print(f"\nEpoch {epoch+1}/{self.config['num_epochs']}")
            train_loss, train_f1_scores = self.train_epoch()
            val_loss, val_metric, val_f1_scores, val_recall_scores = self.validate()

            # Calculate harmonic mean of training F1 scores
            train_f1_values = list(train_f1_scores.values())
            train_f1_mean = len(train_f1_values) / sum(1 / f1 if f1 > 0 else 1e-6 for f1 in train_f1_values)

            print(f"Train Loss: {train_loss:.4f} | Train F1 (Harmonic): {train_f1_mean:.4f}")
            print(f"Val Loss:   {val_loss:.4f} | Val Metric ({self.config['early_stopping_metric']}):   {val_metric:.4f}")

            # Print individual task F1 scores and recall scores
            for task in DATASETS:
                if task in val_f1_scores:
                    print(f"  - {task}: F1 = {val_f1_scores[task]:.4f}, Recall = {val_recall_scores[task]:.4f}")  # recall 출력

            if val_metric > self.best_val_metric:
                self.best_val_metric = val_metric
                self.early_stopping_counter = 0
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'scheduler_state_dict': self.scheduler.state_dict(),
                    'best_metric': self.best_val_metric,  # best_f1 대신 best_metric
                    'config': self.config,
                }, 'best_model.pth')
                print("Saved best model.")
            else:
                self.early_stopping_counter += 1
                print(f"Early stopping counter: {self.early_stopping_counter}/{self.config['early_stopping_patience']}")
                if self.early_stopping_counter >= self.config['early_stopping_patience']:
                    print("Early stopping triggered.")
                    break

# --- Data Augmentation ---
# Data augmentation transforms: consistent 3-channel input
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(size=(28, 28), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomRotation(degrees=(-15, 15)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

val_transforms = transforms.Compose([
    # For validation/test, we usually just resize and normalize
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# --- Load Data, Create Datasets, Model, and Trainer ---
datasets = load_all_datasets(debug=DEBUG)

# Add Grayscale transform *before* ToTensor and Normalize
def get_transforms(train=True):
  transforms_list = []
  if train:
    transforms_list.extend([
        transforms.RandomResizedCrop(size=(28, 28), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
        transforms.RandomRotation(degrees=(-15, 15)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
        transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.2),
    ])
  # else:  # No additional transforms needed for val/test besides Grayscale and Normalize
      # transforms_list.extend([])
  transforms_list.extend([
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels *BEFORE* normalization
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
  ])
  return transforms.Compose(transforms_list)


train_dataset = MedMNISTMultiDataset(datasets, split='train', transform=get_transforms(train=True))
val_dataset = MedMNISTMultiDataset(datasets, split='val', transform=get_transforms(train=False))
test_dataset = MedMNISTMultiDataset(datasets, split='test', transform=get_transforms(train=False))

model = MedMNISTMultiTaskModel(
    backbone_name=config['backbone'],
    pretrained=config['pretrained'],
    head_type=config['head_type'],
    dropout_rate=config['dropout_rate'],
    stochastic_depth_rate=config['stochastic_depth_rate']
)
trainer = Trainer(model, train_dataset, val_dataset, config)

# --- Train the Model ---
trainer.train()

# --- Submission Generation (Example - Adapt as Needed) ---
def load_best_model(checkpoint_path, model, device='cuda'):
    try:
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        best_metric = checkpoint['best_metric'] # best_f1 대신 best_metric
        print(f"Loaded best model from epoch {checkpoint['epoch']} with metric: {best_metric:.4f}") # metric 종류 출력
        return model, best_metric
    except FileNotFoundError:
        print(f"No checkpoint found at {checkpoint_path}")
        return None, 0.0  # Or handle the error as appropriate
    except Exception as e:
        print(f"Error loading checkpoint: {str(e)}")
        return None, 0.0
def create_submission(model, test_dataset, config, device='cuda'):
    if model is None:  # Check if the model is loaded
        print("Cannot create submission: No model loaded.")
        return None

    model.eval()
    all_predictions = []
    test_loader = DataLoader(
        test_dataset, batch_size=config['batch_size'], shuffle=False,
        num_workers=config['num_workers'], pin_memory=True, persistent_workers=True
    )
    task_counters = {task: 0 for task in DATASETS}
    global_id = 0 # global id

    with torch.no_grad(), autocast(enabled=config['use_amp']):
        for batch in tqdm(test_loader, desc='Generating predictions', dynamic_ncols=True):
            images, _, task_ids = batch  # Unpack the batch, ignore labels
            images, task_ids = images.to(device, non_blocking=True), task_ids.to(device, non_blocking=True)
            unique_tasks = torch.unique(task_ids)

            for task_idx in unique_tasks:
                task_name = DATASETS[task_idx]
                # Filter the batch to only include samples from the current task
                mask = task_ids == task_idx
                if mask.any():  # Ensure there are samples for this task
                  task_images = images[mask]
                  features = model.backbone(task_images)
                  outputs = model.heads[task_name](features)
                  preds = outputs.argmax(dim=1).cpu().numpy()
                  n_preds = len(preds)

                  task_start_idx = task_counters[task_name]

                  # Create a list of dictionaries for this batch
                  batch_predictions = [
                      {
                          'id': global_id + i,
                          'label': int(pred),  # Convert to int
                          'task_name': task_name,  # Store task name
                          'id_image_in_task': task_start_idx + i,
                      }
                      for i, pred in enumerate(preds)
                  ]
                  all_predictions.extend(batch_predictions)
                  task_counters[task_name] += n_preds
                  global_id += n_preds


    # Convert to DataFrame and save
    df = pd.DataFrame(all_predictions)
    df = df[['id', 'label', 'task_name', 'id_image_in_task']]  # Ensure correct column order
    df.to_csv('submission.csv', index=False)
    print(f"\nSubmission saved with {len(df)} total predictions")
    return df
# --- Load Best Model and Create Submission ---

model, best_metric = load_best_model('best_model.pth', model)  # best_f1 대신 best_metric
submission_df = create_submission(model, test_dataset, config)

if submission_df is not None:
  print(submission_df.head())

Loading pathmnist...
Loading dermamnist...
Loading octmnist...
Loading pneumoniamnist...
Loading retinamnist...
Loading breastmnist...
Loading bloodmnist...
Loading tissuemnist...
Loading organamnist...
Loading organcmnist...
Loading organsmnist...


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

<ipython-input-2-596e5a40956e>:250: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler(enabled=config['use_amp']) # Use standard GradScaler



Epoch 1/30


Training:   0%|          | 0/859 [00:00<?, ?it/s]<ipython-input-2-596e5a40956e>:305: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.config['use_amp']):
Validating: 100%|██████████| 116/116 [01:19<00:00,  1.46it/s]


...

Train Loss: 0.3164 | Train F1 (Harmonic): 0.7905
Val Loss:   0.6168 | Val Metric (recall):   0.8333
  - pathmnist: F1 = 0.9332, Recall = 0.9334
  - dermamnist: F1 = 0.4524, Recall = 0.4235
  - octmnist: F1 = 0.8985, Recall = 0.8855
  - pneumoniamnist: F1 = 0.9634, Recall = 0.9735
  - retinamnist: F1 = 0.3660, Recall = 0.3612
  - breastmnist: F1 = 0.8711, Recall = 0.8333
  - bloodmnist: F1 = 0.9418, Recall = 0.9443
  - tissuemnist: F1 = 0.6255, Recall = 0.6198
  - organamnist: F1 = 0.9928, Recall = 0.9924
  - organcmnist: F1 = 0.9888, Recall = 0.9887
  - organsmnist: F1 = 0.8540, Recall = 0.8554
Early stopping counter: 4/7


<ipython-input-2-596e5a40956e>:545: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast(enabled=config['use_amp']):


Loaded best model from epoch 25 with metric: 0.8396


Generating predictions: 100%|██████████| 190/190 [00:41<00:00,  4.58it/s]



Submission saved with 96941 total predictions
   id  label  task_name  id_image_in_task
0   0      8  pathmnist                 0
1   1      4  pathmnist                 1
2   2      4  pathmnist                 2
3   3      3  pathmnist                 3
4   4      4  pathmnist                 4
